In [45]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torch.nn as nn

In [2]:
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [4]:
len(words)

32033

In [7]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [109]:
context_window = 3
X, Y = [], []

for w in words[:5]:
    context = [0] * context_window
    print(w)
    for i in w + '.':
        ix = stoi[i]
        X.append(context)
        Y.append(ix)
        print(''.join(itos[i] for i in context), '--->', itos[ix])
        context = context[1:] + [ix]
        
X = torch.tensor(X)      # This is the input part which has all the trigrams i.e. 196113 in a matrix of 196113 x 3
Y = torch.tensor(Y)      # This contains the right index for the predicting char

emma
... ---> e
..e ---> m
.em ---> m
emm ---> a
mma ---> .
olivia
... ---> o
..o ---> l
.ol ---> i
oli ---> v
liv ---> i
ivi ---> a
via ---> .
ava
... ---> a
..a ---> v
.av ---> a
ava ---> .
isabella
... ---> i
..i ---> s
.is ---> a
isa ---> b
sab ---> e
abe ---> l
bel ---> l
ell ---> a
lla ---> .
sophia
... ---> s
..s ---> o
.so ---> p
sop ---> h
oph ---> i
phi ---> a
hia ---> .


In [110]:
X.size(), Y.size()

(torch.Size([32, 3]), torch.Size([32]))

In [111]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([32, 3]), torch.int64, torch.Size([32]), torch.int64)

In [49]:
# <----- DATASET LAOADED ----->
# Time to start with with embeddings and then squeeze the dimension
# Embeddings ---> Representation of inputs, just like GPT uses tokenization

In [50]:
# First embedding then one hot encoding

In [112]:
vocab_size = len(stoi)  # Size of the vocabulary
embedding_dim = 30  # Desired dimensionality of embeddings

embedding_input = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)

embedding_output = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)

In [113]:
embedded_X = embedding_input(X)
embedded_Y = embedding_output(Y) 

In [125]:
embedded_X.size(), embedded_Y.size(), embedded_X.dtype, embedded_Y.dtype
# 196113 - samples, 3 - context-window, 30 - each word represented by vector length 30

(torch.Size([32, 3, 30]), torch.Size([32, 30]), torch.float32, torch.float32)

In [115]:
# g = torch.Generator().manual_seed(2147483647) # for reproducibility
# C = torch.randn((27, 10), generator=g)

W1 = torch.randn((30, 200), generator=g)
b1 = torch.randn(200, generator=g)
W2 = torch.randn((200, 27), generator=g)
b2 = torch.randn(27, generator=g)

parameters = [embedded_X, W1, b1, W2, b2]

In [116]:
sum(p.nelement() for p in parameters) # number of parameters in total

14507

In [117]:
h = torch.tanh(embedded_X @ W1 + b1)

In [118]:
h

tensor([[[-0.9998,  1.0000,  1.0000,  ...,  1.0000,  1.0000,  0.9350],
         [-0.9998,  1.0000,  1.0000,  ...,  1.0000,  1.0000,  0.9350],
         [-0.9998,  1.0000,  1.0000,  ...,  1.0000,  1.0000,  0.9350]],

        [[-0.9998,  1.0000,  1.0000,  ...,  1.0000,  1.0000,  0.9350],
         [-0.9998,  1.0000,  1.0000,  ...,  1.0000,  1.0000,  0.9350],
         [ 0.9725,  0.9998,  0.9038,  ..., -1.0000,  1.0000,  0.9753]],

        [[-0.9998,  1.0000,  1.0000,  ...,  1.0000,  1.0000,  0.9350],
         [ 0.9725,  0.9998,  0.9038,  ..., -1.0000,  1.0000,  0.9753],
         [-0.9893,  1.0000, -0.8249,  ..., -0.9583,  1.0000, -0.9871]],

        ...,

        [[-0.9543,  0.9989,  0.9990,  ...,  1.0000, -0.9633, -1.0000],
         [ 1.0000, -1.0000,  0.3862,  ...,  0.9820, -0.9999, -0.2584],
         [-1.0000, -0.6505, -0.3330,  ..., -0.9847, -0.9989, -1.0000]],

        [[ 1.0000, -1.0000,  0.3862,  ...,  0.9820, -0.9999, -0.2584],
         [-1.0000, -0.6505, -0.3330,  ..., -0.9847, -0.

In [119]:
logits = (h @ W2 + b2)

In [120]:
counts = logits.exp()

In [121]:
probs = counts/counts.sum(1, keepdims=True)

In [122]:
probs.shape

torch.Size([32, 3, 27])

In [ ]:
# < ----------------------------- >

In [131]:
C = torch.randn(27,2)

In [135]:
C[X]

torch.Size([32, 3, 2])

In [139]:
C[X].shape 
# (27, 2) (32, 3) ---> (32, 3, 2)

torch.Size([32, 3, 2])

In [140]:
emb = C[X]
emb.shape
# 2 dimensional embeddings times 3 
# 32 tells about the input

torch.Size([32, 3, 2])

In [145]:
w1 = torch.randn(6,100) # 3 x 2 and 100 weights
b1 = torch.randn(100)

In [149]:
h = torch.tanh(emb.view(-1, 6) @ w1 + b1)

In [151]:
h.shape

torch.Size([32, 100])

In [152]:
w2 = torch.randn(100, 27) 
b2 = torch.randn(27)

In [154]:
logits = h @ w2 + b2
logits

tensor([[ 3.0781e+00, -5.7469e+00,  1.2162e+01, -1.0922e+01,  8.6147e+00,
         -3.9553e+00,  1.0002e+01,  9.7698e+00, -2.9146e+00,  7.0215e+00,
          1.2310e+01,  1.1800e+01,  7.0864e-01,  1.4891e+00, -1.7471e+00,
         -3.8750e-01, -2.3665e+00, -4.0832e+00, -1.4658e+01,  5.0986e+00,
         -3.7365e+00,  7.2896e+00, -7.5972e-01, -9.1084e+00,  6.2999e+00,
          5.8702e+00,  6.9671e+00],
        [ 1.0127e+01,  5.0765e+00,  1.4176e+01, -1.0225e+01,  2.8298e+00,
         -1.4563e+01,  8.6192e+00,  1.5370e+01, -1.2922e+00,  3.6546e+00,
          1.2230e+01, -9.0094e-01,  1.0021e+01,  9.7411e+00, -4.1836e+00,
          1.6294e+00, -1.5708e+00,  2.6760e+00, -8.5778e+00,  1.7712e+01,
          3.2870e+00,  6.1405e+00, -1.3942e+00, -4.7539e+00,  3.4964e-01,
          3.1095e+00,  1.7827e+01],
        [ 1.0015e+01,  3.8588e-01, -8.5830e-02, -5.0984e+00, -5.6510e+00,
         -3.1822e+00,  3.2991e+00,  1.6711e+01,  2.2756e-01, -4.0187e+00,
          1.2676e+01,  1.2180e+00,  1.19

In [155]:
counts = logits.exp()

In [157]:
probs = counts / counts.sum(1, keepdims=True)

In [158]:
probs.shape

torch.Size([32, 27])

In [159]:
Y

tensor([ 5, 13, 13,  1,  0, 15, 12,  9, 22,  9,  1,  0,  1, 22,  1,  0,  9, 19,
         1,  2,  5, 12, 12,  1,  0, 19, 15, 16,  8,  9,  1,  0])

In [161]:
loss = -probs[torch.arange(32), Y].log().mean()
loss

tensor(13.7183)